Import needed libraries.

In [2]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import gregw

/Users/gregwinter/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Bring in train and test data.

In [3]:
train = pd.read_csv("house_prices_train.csv")
test = pd.read_csv("housing_prices_test.csv")

Cut out the sale price of the train data.

In [4]:
y = train['SalePrice']
train.drop(['SalePrice'], axis = 1, inplace = True)

Copy and cut out ID of train data.

In [6]:
test_id = test['Id']
alldata = pd.concat([train, test], axis=0)
alldata.drop(['Id'], axis = 1, inplace = True)

Change the dtype of some of the columns.

In [7]:
alldata['MSSubClass'] = alldata['MSSubClass'].astype(object)
alldata['MoSold'] = alldata['MoSold'].astype(object)
alldata['YrSold'] = alldata['YrSold'].astype(object)
alldata['BsmtFullBath'] = alldata['BsmtFullBath'].astype(object)
alldata['BsmtHalfBath'] = alldata['BsmtHalfBath'].astype(object)

Fill in simple missing values.

In [8]:
alldata.LotFrontage = alldata.LotFrontage.fillna(0)
alldata['GarageCars'] = alldata['GarageCars'].fillna(0)
alldata['GarageArea'] = alldata['GarageArea'].fillna(519)
alldata.Alley = alldata.Alley.fillna('None')
alldata.FireplaceQu = alldata.FireplaceQu.fillna('None')
alldata.GarageType = alldata.GarageType.fillna('None')
alldata['GarageYrBlt'] = alldata['GarageYrBlt'].fillna('None')
alldata['GarageFinish'] = alldata['GarageFinish'].fillna('None')
alldata['GarageQual'] = alldata['GarageQual'].fillna('None')
alldata['GarageCond'] = alldata['GarageCond'].fillna('None')
alldata['PoolQC'] = alldata['PoolQC'].fillna('None')
alldata['Fence'] = alldata['Fence'].fillna('None')
alldata['MiscFeature'] = alldata['MiscFeature'].fillna('None')

Impute values for missing values.

In [9]:
gregw.impute_cal('MSZoning', alldata)
gregw.impute_cal('Utilities', alldata)
gregw.impute_cal('Exterior1st', alldata)
gregw.impute_cal('Exterior2nd', alldata)
gregw.impute_cal('MasVnrType', alldata)
gregw.impute_reg('MasVnrArea', alldata)
gregw.impute_cal('BsmtQual', alldata)
gregw.impute_cal('BsmtCond', alldata)
gregw.impute_cal('BsmtExposure', alldata)
gregw.impute_cal('BsmtFinType1', alldata)
gregw.impute_reg('BsmtFinSF1', alldata)
gregw.impute_cal('BsmtFinType2', alldata)
gregw.impute_cal('Electrical', alldata)
gregw.impute_cal('BsmtFullBath', alldata)
gregw.impute_cal('BsmtHalfBath', alldata)
gregw.impute_cal('KitchenQual', alldata)
gregw.impute_cal('Functional', alldata)
gregw.impute_reg('GarageCars', alldata)
gregw.impute_cal('SaleType', alldata)

gregw.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred.drop(col1, axis = 1, inplace = True)
gregw.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train.drop(col1, axis = 1, inplace = True)
gregw.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train.drop(col1, axis = 1, inplace = True) # remove y from train set
gregw.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.py

No Missing / NaN Data


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,0,None,None,None,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,0,None,None,None,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,0,None,None,None,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,0,None,None,None,0,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,Inside,...,0,0,None,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,8,2007,WD,Normal
7,60,RL,0.0,10382,Pave,None,IR1,Lvl,AllPub,Corner,...,0,0,None,None,Shed,350,11,2009,WD,Normal
8,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,4,2008,WD,Abnorml
9,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,Corner,...,0,0,None,None,None,0,1,2008,WD,Normal


Check for anything I may of missed.

In [10]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 79 columns):
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 n

Create dummy values.

In [11]:
dummies_data = pd.get_dummies(alldata,drop_first=True)

Split data back into the orginal train and test data.

In [12]:
train = dummies_data.iloc[:1460]
test = dummies_data.iloc[1460:]

Create a baseline using the default paramters of XGBoost.

In [240]:
cross_val_score(XGBRegressor(), train, y, cv = 3).mean()

0.88517104064583563

Create dictionary of paramaters to be tuned and cross validated.

In [241]:
parameters = {'base_score':[.50, .55, .60], 'max_depth': [1,2,3], 'n_estimators':[1006, 1008, 1010],
              'min_child_weight' :[1], 'colsample_bytree' :[.79, .8, .81],
              'colsample_bylevel':[0.79, .8, .81]}

In [ ]:
XGBR = GridSearchCV(XGBRegressor(), parameters, n_jobs=1, cv=3, verbose=2)
XGBR.fit(train, y)

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1006, base_score=0.5, max_depth=1 
[CV]  colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1006, base_score=0.5, max_depth=1, total=   4.8s
[CV] colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1006, base_score=0.5, max_depth=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s


[CV]  colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1006, base_score=0.5, max_depth=1, total=   4.7s
[CV] colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1006, base_score=0.5, max_depth=1 
[CV]  colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1006, base_score=0.5, max_depth=1, total=   5.1s
[CV] colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1008, base_score=0.5, max_depth=1 
[CV]  colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1008, base_score=0.5, max_depth=1, total=   5.3s
[CV] colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1008, base_score=0.5, max_depth=1 
[CV]  colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1008, base_score=0.5, max_depth=1, total=   5.2s
[CV] colsample_bytree=0.79, colsample_bylevel=0.79, min_child_weight=1, n_estimators=1008, base_scor

In [227]:
print 'Best score for data:', XGBR.best_score_
XGBR.best_estimator_ 

Best score for data: 0.904699446554


XGBRegressor(base_score=0.55, colsample_bylevel=0.8, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=1008, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [228]:
tuned_final = XGBRegressor(base_score=0.5, colsample_bylevel=0.8, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=1008, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1).fit(train, y)

In [234]:
answer = pd.DataFrame()
answer['SalePrice'] = tuned_final.predict(test)

In [236]:
final = pd.concat([test_id, answer['SalePrice']], axis=1)

In [238]:
final.to_csv('kaggle_house_prices.csv', index=False)